In [1]:
!pip install manuscript-ocr==0.1.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.

In [2]:
import requests, zipfile, io


# 2) Скачиваем архив с Яндекс.Диска и распаковываем
share_link = "https://disk.yandex.ru/d/tUFCt2X6iWPS_w"
api_url    = "https://cloud-api.yandex.net/v1/disk/public/resources/download"

# получаем прямую ссылку
download_url = requests.get(api_url, params={"public_key": share_link}).json()["href"]

# загружаем и сразу распаковываем в /content/data
resp = requests.get(download_url)
with open("/content/data.zip", "wb") as f:
    f.write(resp.content)

with zipfile.ZipFile("/content/data.zip", "r") as z:
    z.extractall("/content/data")

In [3]:
import os
from manuscript.detectors.east import train

base = "/content/data"
train_images, train_anns = [], []
val_images, val_anns = [], []

# Автопоиск всех train/test директорий и аннотаций
for root, dirs, files in os.walk(base):
    if "train.json" in files and "train_images" in dirs:
        train_images.append(os.path.join(root, "train_images"))
        train_anns.append(os.path.join(root, "train.json"))
    if "test.json" in files and "test_images" in dirs:
        val_images.append(os.path.join(root, "test_images"))
        val_anns.append(os.path.join(root, "test.json"))

# Проверка найденных путей
print("Train:", train_images, train_anns)
print("Val:  ", val_images, val_anns)

# Запуск обучения
best_model = train(
    train_images=train_images,
    train_anns=train_anns,
    val_images=val_images,
    val_anns=val_anns,
    target_size=256,
    epochs=1,
    batch_size=4,
    use_sam=False,
    freeze_first=False,
)
print("Сохранён чекпоинт:", best_model)

Train: ['/content/data/Архивы (формат для обучения детектора)/train_images'] ['/content/data/Архивы (формат для обучения детектора)/train.json']
Val:   ['/content/data/Архивы (формат для обучения детектора)/test_images'] ['/content/data/Архивы (формат для обучения детектора)/test.json']
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth


100%|██████████| 171M/171M [00:02<00:00, 80.8MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/manuscript/detectors/east/train_utils.py:88: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Train 1:   0%|          | 0/383 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/manuscript/detectors/east/train_utils.py:154: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Сохранён чекпоинт: TextDetectionFCN(
  (backbone): ResNetFeatureExtractor(
    (extractor): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Module(
        (0): Module(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=